In [1]:
from argparse import ArgumentParser, ArgumentDefaultsHelpFormatter
import numpy as np

import sys
sys.path.insert(0,'../../')

from rcalgo.tf.utils.text_utils import *
import pickle as pkl
from rcalgo.tf.tfmodels.texar_models.tx_rnnlm import *
#import texar as tx
from scipy.stats import rv_discrete
from rcalgo.tf.tfmodels.rnnlm import *
from rcalgo.tf.utils.tf_func import inverse_sigmoid_sample_proba
import texar.tf as tx


In [2]:
with open('tmp_photo_comments_sentences.pkl','rb') as f:
    sentences = pkl.load(f)

In [3]:
word_freq = word_count(sentences, word_seg=False)
term_id_map = build_word2idx(word_freq, freq_threshold=2)
id_term_map = dict([(v,k) for k, v in term_id_map.items()]) 


In [4]:
from rcalgo.tf.utils.train_config import *

args = parse_initial_training_args(
    [
        '--learning-rate', '0.001',
        '--n-epochs', '2',
        '--opt', 'adam',
        '--batch-size', '512',
        '--p-coef', '0',
        '--keep-prob','1.0',
        '--embedding-size', '256',
        '--model-dir', 'model/',
        '--model-name', 'rnnlm',
        '--num-layers', '2',
        '--gpu-id', '0',
        '--gpu-num', '2',
        '--hidden-size','256',
        '--maxlen', '100',
        '--position-type', '2,3,4,5',
        '--epoch-save',
        '--use-residual',
        #'--state-init',
        '--beam-width', '200',
        '--sample-decay-factor', '500000',
       # '--sampling-probability', '0.01',
      #  '--epoch-save',
    ]
)
#args.beam_width = 5
args.sos_flag = True
args.expand_input = True
args.sampled_softmax = False

In [5]:
res = {}
graph_to_use = tf.Graph()
config = tf.ConfigProto(allow_soft_placement=True)
config.gpu_options.allow_growth=True
rnnlm_model = TxRNNLMModel(args, graph_to_use, word_dict=term_id_map)
rnnlm_model.build_model()
rnnlm_model.build_model_summary()
display(rnnlm_model.model_summary())
rnnlm_model.run([sentences], test_size=0.5)

/home/web_server/anaconda3/lib/python3.6/site-packages/tensorflow/python/ops/gradients_util.py:93: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/home/web_server/anaconda3/lib/python3.6/site-packages/tensorflow/python/ops/gradients_util.py:93: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


total parameter number: 5243117


variable_name variable_shape parameters
0    gpu/embedding/embedding_layer/embedding_table:0    [8172, 256]    2092032
1  gpu/basic_rnn_decoder/multi_rnn_cell/cell_0/ls...    [512, 1024]     524288
2  gpu/basic_rnn_decoder/multi_rnn_cell/cell_0/ls...         [1024]       1024
3  gpu/basic_rnn_decoder/multi_rnn_cell/cell_1/ls...    [512, 1024]     524288
4  gpu/basic_rnn_decoder/multi_rnn_cell/cell_1/ls...         [1024]       1024
5               gpu/basic_rnn_decoder/dense/kernel:0    [256, 8173]    2092288
6                 gpu/basic_rnn_decoder/dense/bias:0         [8173]       8173

total number of epochs: 2
tasks: [<tftraining.incremental_training.Task object at 0x7f182bbda438>]
lr values: [0.001]


epoch 0 train time(sec): 214.0900890827179


epoch 0 test time(sec): 118.08378887176514
Model checkpoint saved in file: model/rnnlm-epoch0.ckpt
Model checkpoint saved in file: model/rnnlm.ckpt
{'best_train_metric': 0.3836,
 'best_validate_metric': 0.4119,
 'epoch_index': 0,
 'train_loss': 5.1781,
 'train_metric': 0.3836,
 'validate_loss': 4.7614,
 'validate_metric': 0.4119}
lr values: [0.001]


epoch 1 train time(sec): 206.74733352661133


epoch 1 test time(sec): 119.77875256538391
Model checkpoint saved in file: model/rnnlm-epoch1.ckpt
Model checkpoint saved in file: model/rnnlm.ckpt
{'best_train_metric': 0.4225,
 'best_validate_metric': 0.4274,
 'epoch_index': 1,
 'train_loss': 4.6176,
 'train_metric': 0.4225,
 'validate_loss': 4.6679,
 'validate_metric': 0.4274}


In [17]:
with rnnlm_model.job.session.graph.as_default():
    input_var, state_var, tmp_state_var = tf.get_collection('input_dict')
    go_value = tf.get_collection('go_value')[0]
    outputs = tf.get_collection('output_dict')[0:5]
    greedy_logits, greedy_sample_id, sample_logits, sample_sample_id, beam_search_results = outputs
    sos_flag = tf.get_collection('sos_flag')[0]
    eos_flag = tf.get_collection('eos_flag')[0]
    top_k = tf.get_collection('top_k_sample')[0]
    softmax_temperature = tf.get_collection('softmax_temperature')[0]
    final_states, logits = tf.get_collection('output_dict')[6:]

In [21]:
f1, t1 = rnnlm_model.job.session.run([final_states, logits],
                            feed_dict={input_var: ['傻'], state_var: np.zeros(shape=(1,4, 256)), eos_flag: False})
f2, t2 = rnnlm_model.job.session.run([final_states, logits], 
                                     feed_dict={input_var: ['逼'], tmp_state_var: f1,
                                                sos_flag: False, eos_flag: True})
f3, t3 = rnnlm_model.job.session.run([final_states, logits],
                                     feed_dict={input_var: ['傻逼'], state_var: np.zeros(shape=(1,4,256)), eos_flag: True})
print(f2-f3)

In [20]:
xxx = rnnlm_model.job.session.run([sample_sample_id], feed_dict={input_var: ['我'],
                                                           sos_flag: True,
                                                           eos_flag: False,
                                                           softmax_temperature: [0.9],
                                                           top_k: 10})

In [24]:
def out_print(num_arr, id_term_map, eos=rnnlm_model.nb_words - 1):
    #print(num_arr)
    text = ''
    for i in num_arr:
        if i == eos:
            break
        else:
            text += id_term_map[i]
    return text

In [33]:
samples = rnnlm_model.job.session.run(beam_search_results, feed_dict={input_var: ['']})
print(samples.shape)
for i in range(20):
    text = out_print(samples[0,:,i], id_term_map)
    print(text)

(10, 3, 200)

不
好
美
我
哥
老
大
没
你
哦
偶
宝
小
真
一
1
中
安
头


In [26]:
state = rnnlm_model.job.session.run(final_states, 
                                    feed_dict={input_var: ['我'], 
                                               eos_flag: False})

In [27]:
samples = rnnlm_model.job.session.run(beam_search_results,
                                      feed_dict={input_var: [''],
                                                 state_var: state,
                                                 #sos_flag: False, 
                                                 go_value: [term_id_map['爱']]})

In [28]:
for i in range(200):
    text = out_print(samples[0,:,i], id_term_map)
    print('我爱'+text)

我爱
我爱，
我爱的
我爱你
我爱了
我爱我
我爱不


KeyError: 8143